# Julia: explore type hierarchy

Shortly, we will use these two functions to explore Julia's type-system.

In [1]:
function printsub(T::Union{UnionAll,DataType}; first_iter = true, last=false, padding="")
    if first_iter
        println(T)
        # don't print "| "
        last=true
        new_padding = ""
    else
        if last 
            println(padding * "┗ ",T)
            new_padding = padding * "  "
        else
            println(padding * "┣ ",T)
            new_padding = padding *  "┃ "
        end
    end
    
    st = subtypes(T)
        
    if ! isempty(st)
        for sub in st[1:lastindex(st)-1] 
            printsub(sub, first_iter=false, padding=new_padding)
        end
        printsub(st[end], first_iter=false, last=true, padding=new_padding)
    end
end

function printsuper(T::Union{UnionAll,DataType}, res=T)
    if supertype(T) != Any
        printsuper(supertype(T), "$(supertype(T)) -> $res")
    else
        println("Any -> $res")
    end
end;


To summarize some of the most important things about Julia's type system (for more refer to [Julia docs](https://docs.julialang.org/en/v1/manual/types/index.html)):
- it is dynamic, but you can indicate that certain values are of specific types
- hierarchical relationships are explicitly declared
    - all concrete types may only have abstract types as their supertypes
    - abstract and concrete types can be parameterized by other types
    - abstract types cannot be instantiated
- all values in Julia are true objects having a type
- the only type a value has is its actual type when the program is running
- only values, not variables, have types
- a primitive type is a concrete type whose data consists of plain old bits
- a composite type is a collection of named fields
- functions are not bundled with the objects they operate on
    - the types of all of a function's arguments are considered when selecting a method, rather than just the first one
    
Furthermore, types in Julia are objects, this implies that we may use functions on them. 
We will be using these build-in functions:
- `isa(x, type)`, to test if x is of the given type
- `typeof(x)`, to get the type of x
- `supertype(T::UnionAll)`, to get the supertype of the UnionAll T
- `supertype(T::DataType)`, to get the supertype of the DataType T
- `subtypes(T::UnionAll)`, to get a list of imediate subtypes of UnionAll T
- `subtypes(T::DataType)`, to get a list of imediate subtypes of DataType T
- `isconcretetype(T)`, to determine wheter type T is a concrete type

The interesting question is what types have the types that are now viewed as objects. Based on the documentation, experimentation and my assumptions we have several types of types.
- `DataType`
    - a type of all abstract, primitive, composite types
- `Union`
    - a type of all types constructed using the Union keyword
- `UnionAll`
    - a type of parametric types
- `Core.TypeofBottom`
    - the type of Union{}

Maybe you are asking what the types of `DataType`, `Union`, `UnionAll` and `Core.TypeofBottom` are. They are all of the type `DataType`, as we can quickly check.

In [2]:
typeof.([DataType, Union, UnionAll, Core.TypeofBottom])

4-element Array{DataType,1}:
 DataType
 DataType
 DataType
 DataType

Moreover, there is a special parametric abstract type called `Type`, which stands for `Type{N} where N`. It has all type objects as its instances and as a parametric type, its type is UnionAll. By the way, the its subtype hierarchy looks like this.

In [3]:
printsub(Type{N} where N)

Type{N} where N
┣ Core.TypeofBottom
┣ DataType
┣ Union
┗ UnionAll


And these types of types that are listed above are all contrete types i.e. they can be instantiated.

In [4]:
isconcretetype.([DataType, Union, UnionAll, Core.TypeofBottom])

4-element BitArray{1}:
 1
 1
 1
 1

There is a very special thing about `Type{N} where N` after all. 

Given a type `T`, we can construct `Type{T}` which behaves as an abstract type with exactly one instance (object of this kind), the type `T` itself. Put differently, for some specific `T` is `Type{T}` an object of the type `DataType`. The only value a object of the type `Type{T}` can contain is the type `T`.

`Type{T}` is defined by the following property: `isa(A,Type{B})` is true if and only if A and B are the same object and that object is a type. (As it is written in the documentation.)

These types are called singleton types and you can read more about them [here](https://docs.julialang.org/en/v1/manual/types/#man-singleton-types-1).

All types are implicitly subtypes of `Any`. And `Any` is of the type `DataType`. Of course `Type{N} where N`, shortly called just `Type` is also a subtype of `Any` and so is also `DataType`.

In [5]:
length(subtypes(Any))

446

In [6]:
subtypes(Any)[1:20]

20-element Array{Any,1}:
 AbstractArray
 AbstractChannel
 AbstractChar
 AbstractDict
 AbstractDisplay
 AbstractSet
 AbstractString
 Any
 Base.AbstractCartesianIndex
 Base.AbstractCmd
 Base.AbstractLock
 Base.ArithmeticStyle
 Base.AsyncCollector
 Base.AsyncCollectorState
 Base.AsyncCondition
 Base.AsyncGenerator
 Base.AsyncGeneratorState
 Base.BaseDocs.Keyword
 Base.BottomRF
 Base.Broadcast.BitMaskedBitArray

So, let's look at some more examples.

In [7]:
# syntax for UnionAll types allows to omit parameters
Array == Array{T, N} where {T,N}

true

In [8]:
typeof(Array)

UnionAll

In [9]:
typeof(Array{Int64})

UnionAll

In [10]:
typeof(Array{Int64,2})

DataType

In [11]:
printsuper(Array)

Any -> AbstractArray{T,N} where N where T -> DenseArray{T,N} where N where T -> Array


In [12]:
# Int is an alias for Int64 (depends on arch.)
printsuper(Array{Int,2})

Any -> AbstractArray{Int64,2} -> DenseArray{Int64,2} -> Array{Int64,2}


In [13]:
printsub(DenseArray{T,N} where {T,N})

DenseArray{T,N} where N where T
┣ Array
┣ Base.CodeUnits
┣ Base.Experimental.Const
┣ Random.UnsafeView
┣ SharedArrays.SharedArray
┗ SuiteSparse.CHOLMOD.Dense


In [14]:
# Only declared types (DataType) have unambiguous supertypes.
printsuper(Array{Int64,4})

Any -> AbstractArray{Int64,4} -> DenseArray{Int64,4} -> Array{Int64,4}


How are numbers structured in Julia?

In [15]:
printsub(Number)

Number
┣ Complex
┗ Real
  ┣ AbstractFloat
  ┃ ┣ BigFloat
  ┃ ┣ Float16
  ┃ ┣ Float32
  ┃ ┗ Float64
  ┣ AbstractIrrational
  ┃ ┗ Irrational
  ┣ Integer
  ┃ ┣ Bool
  ┃ ┣ Signed
  ┃ ┃ ┣ BigInt
  ┃ ┃ ┣ Int128
  ┃ ┃ ┣ Int16
  ┃ ┃ ┣ Int32
  ┃ ┃ ┣ Int64
  ┃ ┃ ┗ Int8
  ┃ ┗ Unsigned
  ┃   ┣ UInt128
  ┃   ┣ UInt16
  ┃   ┣ UInt32
  ┃   ┣ UInt64
  ┃   ┗ UInt8
  ┗ Rational


In [16]:
printsub(AbstractString)

AbstractString
┣ String
┣ SubString
┣ SubstitutionString
┗ Test.GenericString


In [17]:
typeof(AbstractArray{String,1})

DataType

In [18]:
printsub(AbstractArray{String,1})

AbstractArray{String,1}
┣ AbstractRange{String}
┃ ┣ LinRange{String}
┃ ┣ OrdinalRange{String,S} where S
┃ ┃ ┣ AbstractUnitRange{String}
┃ ┃ ┗ StepRange{String,S} where S
┃ ┗ StepRangeLen{String,R,S} where S where R
┣ Base.LogicalIndex{String,A} where A<:(AbstractArray{Bool,N} where N)
┣ Base.ReinterpretArray{String,1,S,A} where A<:AbstractArray{S,1} where S
┣ Base.ReshapedArray{String,1,P,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where P<:AbstractArray
┣ Core.Compiler.AbstractRange{String}
┃ ┣ Core.Compiler.LinRange{String}
┃ ┣ Core.Compiler.OrdinalRange{String,S} where S
┃ ┃ ┣ Core.Compiler.AbstractUnitRange{String}
┃ ┃ ┗ Core.Compiler.StepRange{String,S} where S
┃ ┗ Core.Compiler.StepRangeLen{String,R,S} where S where R
┣ DenseArray{String,1}
┃ ┣ Array{String,1}
┃ ┣ Base.CodeUnits{String,S} where S<:AbstractString
┃ ┣ Base.Experimental.Const{String,1}
┃ ┣ Random.UnsafeView{String}
┃ ┗ SharedArrays.SharedArray{String,1}
┣ JSO

Functions in Julia are first-class objects.

In [19]:
printsuper(Function)

Any -> Function


In [20]:
isa(+, Function)

true

Thanks for reading.